In [122]:
import pandas as pd
import plotly.express as px

In [123]:
org_data = pd.read_parquet("DSCB310 - UE2 - Shopping Carts.parquet")

In [124]:
org_data = org_data.reset_index(drop= True)

In [125]:
df = org_data.copy()

#### **order_size**

In [126]:
df_order_size = df.groupby(["order_id"], as_index= False).agg(order_size = ("add_to_cart_order", "max"))

In [140]:
ordersize_bins = [1, 3, 5, 10, 15, 20, 55, 75, 100, 125]
df_order_size["order_size_cut"] = pd.cut(df_order_size.order_size, bins= ordersize_bins, include_lowest= False)
df_order_size["order_size_cut"] = df_order_size.order_size_cut.astype("str")

In [139]:
df_order_size.order_size.unique()

array([  8,   3,   5,   2,   7,  10,  11,  16,   4,  14,  49,  34,  21,
         6,  25,   9,   1,  20,  15,  13,  23,  22,  12,  18,  24,  27,
        36,  29,  19,  17,  39,  26,  52,  28,  31,  45,  38,  30,  32,
        35,  43,  33,  53,  65,  37,  44,  41,  42,  40,  54,  47,  48,
        57,  46,  56,  64,  50, 127,  83,  61,  92,  58,  63,  95,  62,
        59,  51,  66,  55,  60,  99,  91,  96,  81,  70,  76,  67,  68,
        72,  98,  82,  73,  79,  69,  87,  71,  88, 145,  86,  78,  80,
        94,  93, 108,  74,  77, 109, 105, 100,  89])

In [128]:
df = df.merge(df_order_size, how= "left", on= "order_id")

#### **order_per_user**

In [129]:
df_orders_per_user = df.groupby(["user_id"], as_index= False).agg(orders= ("order_id", "nunique"))

In [130]:
orders_bins = [1, 5, 10, 15, 20, 25, 50, 70, 90, 110]
df_orders_per_user["orders_cut"] = pd.cut(df_orders_per_user.orders, bins= orders_bins, include_lowest= False)
df_orders_per_user["orders_cut"] = df_orders_per_user.orders_cut.astype("str")
df_orders_per_user.sort_values(by="orders_cut", ascending= True)

,user_id,orders,orders_cut
20293,112001,4,"(1, 5]"
27275,150947,4,"(1, 5]"
7133,39572,5,"(1, 5]"
27278,150957,5,"(1, 5]"
7136,39584,5,"(1, 5]"
...,...,...,...
8792,48897,99,"(90, 110]"
1406,7713,94,"(90, 110]"
16259,89724,100,"(90, 110]"
33689,186006,100,"(90, 110]"


In [131]:
df_orders_per_users = df_orders_per_user.drop(df_orders_per_user.columns[1], inplace= True, axis= 1)

In [133]:
df = df.merge(df_orders_per_user, how= "left", on= "user_id")

#### **days_since_prior_order**

In [150]:
df.loc[(df.days_since_prior_order.isna())]

,order_id,product_id,add_to_cart_order,reordered,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,tip,product_name,aisle_id,department_id,department,aisle,county,order_size,order_size_cut,orders_cut
71,13,46383,15,0,106387,1,5,19,NaN,0,Imported Capote Capers,110,13,pantry,pickled goods olives,San Mateo,16,"(15, 20]","(1, 5]"
72,13,21019,14,0,106387,1,5,19,NaN,0,Roasted Red Pepper Hummus,67,20,deli,fresh dips tapenades,San Mateo,16,"(15, 20]","(1, 5]"
73,13,33768,7,0,106387,1,5,19,NaN,0,Sinfully Sweet Campari Tomatoes,83,4,produce,fresh vegetables,San Mateo,16,"(15, 20]","(1, 5]"
74,13,25195,16,0,106387,1,5,19,NaN,0,Roasted Garlic Loaf,112,3,bakery,bread,San Mateo,16,"(15, 20]","(1, 5]"
75,13,40709,8,0,106387,1,5,19,NaN,0,Sea Salt & Vinegar Potato Chips,107,19,snacks,chips pretzels,San Mateo,16,"(15, 20]","(1, 5]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6132445,606893,38768,2,0,92134,1,5,10,NaN,0,Sweet Kale Salad Mix,32,4,produce,packaged produce,Stanislaus,2,"(1, 3]","(25, 50]"
6132665,606915,27966,1,0,107102,1,3,10,NaN,0,Organic Raspberries,123,4,produce,packaged vegetables fruits,Amador,4,"(3, 5]","(25, 50]"
6132666,606915,27845,4,0,107102,1,3,10,NaN,0,Organic Whole Milk,84,16,dairy eggs,milk,Amador,4,"(3, 5]","(25, 50]"
6132667,606915,26940,3,0,107102,1,3,10,NaN,0,Organic Large Green Asparagus,83,4,produce,fresh vegetables,Amador,4,"(3, 5]","(25, 50]"


In [134]:
df.groupby(["days_since_prior_order"], as_index= False).agg(orders= ("order_id", "nunique"))

,days_since_prior_order,orders
0,0.0,12005
1,1.0,26178
2,2.0,34983
3,3.0,38541
4,4.0,39479
5,5.0,38254
6,6.0,42605
7,7.0,57068
8,8.0,32510
9,9.0,21219


#### df

In [136]:
df.to_csv("orderdata.csv")